# Backpropagation in Affine(Linear) Layer
---

## References
* Great math notebook by 조준우 http://nbviewer.jupyter.org/github/metamath1/ml-simple-works/blob/master/fitting/matrix-derivative.ipynb
* [Slider Share in Sungbin Lim, Mathematical Scientist @ Kakao Brain](https://www.slideshare.net/ssuser7e10e4/matrix-calculus)

In [1]:
import numpy as np
np.random.seed(123)

Before do it, have to learn two concepts below to solve our problem.

### Kronecker product

There are two matrix, $A: p \times q$ and $B: r \times s$, Kronecker product of $A$ and $B$ will have a shape of $pr \times qs$.

$$A \otimes B = \{ a_{ij}B \}$$

For example,

$$A = \begin{bmatrix} \color{RoyalBlue}{a_{11}} & \color{OrangeRed}{a_{12}} \\ \color{YellowGreen}{a_{21}} & \color{Goldenrod}{a_{22}} \end{bmatrix}, B = \begin{bmatrix} b_{11} & b_{12} \\ b_{21} & b_{22} \\ b_{31} & b_{32} \end{bmatrix}$$

$$\begin{aligned}
A \otimes B &= \begin{bmatrix} \color{RoyalBlue}{a_{11}}B & \color{OrangeRed}{a_{12}}B \\ \color{YellowGreen}{a_{21}}B & \color{Goldenrod}{a_{22}}B \end{bmatrix} \\
&= 
\left[ \begin{array}{cc:cc}
\color{RoyalBlue}{a_{11}}b_{11} & \color{RoyalBlue}{a_{11}}b_{12} & \color{OrangeRed}{a_{12}}b_{11} & \color{OrangeRed}{a_{12}}b_{12} \\
\color{RoyalBlue}{a_{11}}b_{21} & \color{RoyalBlue}{a_{11}}b_{22} & \color{OrangeRed}{a_{12}}b_{21} & \color{OrangeRed}{a_{12}}b_{22} \\
\color{RoyalBlue}{a_{11}}b_{31} & \color{RoyalBlue}{a_{11}}b_{32} & \color{OrangeRed}{a_{12}}b_{31} & \color{OrangeRed}{a_{12}}b_{32} \\ \hdashline
\color{YellowGreen}{a_{21}}b_{11} & \color{YellowGreen}{a_{21}}b_{12} & \color{Goldenrod}{a_{22}}b_{11} & \color{Goldenrod}{a_{22}}b_{12} \\
\color{YellowGreen}{a_{21}}b_{21} & \color{YellowGreen}{a_{21}}b_{22} & \color{Goldenrod}{a_{22}}b_{21} & \color{Goldenrod}{a_{22}}b_{22} \\
\color{YellowGreen}{a_{21}}b_{31} & \color{YellowGreen}{a_{21}}b_{32} & \color{Goldenrod}{a_{22}}b_{31} & \color{Goldenrod}{a_{22}}b_{32} \\
\end{array} \right]
\end{aligned}$$

In [2]:
A = np.random.randint(1, 10, size=(2, 2))
B = np.random.randint(1, 10, size=(3, 2))
print('===A===')
print(A)
print('===B===')
print(B)
print('===AoB===')
print(np.kron(A, B))

===A===
[[3 3]
 [7 2]]
===B===
[[4 7]
 [2 1]
 [2 1]]
===AoB===
[[12 21 12 21]
 [ 6  3  6  3]
 [ 6  3  6  3]
 [28 49  8 14]
 [14  7  4  2]
 [14  7  4  2]]


### Vec & Vec Transformation

In Linear Algebra, the vectorization of a matrix is a linear transformation which converts the matrix into a column vector. 

For a $n \times m$ matrix $A$,

$$A = \begin{bmatrix} 
a_{11} & a_{12} & \cdots & a_{1m} \\
a_{21} & a_{22} & \cdots & a_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nm} 
\end{bmatrix} = ( a_{ij} ) \in \Bbb{R}^{n \times m}$$

$$Vec(A) = [a_{11}, a_{12}, \cdots, a_{1m}, a_{21}, a_{22}, \cdots, a_{2m}, a_{n1}, a_{n2}, \cdots , a_{nm}]^T$$

Vec Transformation is the general form of Transfomation:

$$A^{(1)} = \begin{bmatrix} 
a_{11} & a_{21} & \cdots & a_{n1} \\
a_{12} & a_{22} & \cdots & a_{n2} \\
\vdots & \vdots & \ddots & \vdots \\
a_{1m} & a_{2m} & \cdots & a_{mn} 
\end{bmatrix}= ( a_{ij} ) \in \Bbb{R}^{m \times n}$$

$$A^{(2)} = \left[ \begin{array}{c:c}
a_{11} & a_{31} & \cdots & a_{(n-1)1} \\
a_{21} & a_{41} & \cdots & a_{n1} \\ \hdashline
a_{12} & a_{32} & \cdots & a_{(n-1)2} \\
a_{22} & a_{42} & \cdots & a_{n2} \\ \hdashline
\vdots & \vdots & \ddots & \vdots \\ \hdashline
a_{1m} & a_{3m} & \cdots & a_{(n-1)m} \\ 
a_{2m} & a_{4m} & \cdots & a_{nm} 
\end{array} \right]= ( a_{ij} ) \in \Bbb{R}^{2m \times n/2}$$

* $k$ must be a natural number that can be devide row number ($n$) in the matrix ($A$)

In [3]:
def vec(x):
    dim = np.prod(x.shape)
    vec_x = x.reshape((dim, 1), order='F')
    return vec_x

In [8]:
def vec_transpose(X, k):
    r, c = X.shape
    assert isinstance(k, int), 'k must be int type'
    assert r % k == 0, 'k must be a natural number that can divede matrix row number'
    new_r, new_c = k*c, int(r/k)
    Y = np.zeros((new_r, new_c), dtype=X.dtype)
    
    it = np.nditer(X, flags=['multi_index'], op_flags=['readwrite'], order='F')
    while not it.finished:
        r_ = it.multi_index[0] % k + it.multi_index[1] * k
        c_ = int(it.multi_index[0]/k)
        Y[r_, c_] = X[it.multi_index]
        it.iternext()
    return Y

In [15]:
A = np.random.randint(0, 10, size=(9, 4))
A

array([[6, 6, 1, 3],
       [4, 3, 1, 0],
       [5, 8, 6, 8],
       [9, 1, 0, 3],
       [1, 3, 4, 7],
       [6, 1, 4, 3],
       [3, 7, 6, 8],
       [6, 4, 4, 7],
       [0, 0, 9, 8]])

In [18]:
vec_transpose(A, 3)

array([[6, 9, 3],
       [4, 1, 6],
       [5, 6, 0],
       [6, 1, 7],
       [3, 3, 4],
       [8, 1, 0],
       [1, 0, 6],
       [1, 4, 4],
       [6, 4, 9],
       [3, 3, 8],
       [0, 7, 7],
       [8, 3, 8]])

## Object

First let's define 3 by 1(or 1 by 3) vector ($x$) and 3 by 2 matrix ($W$)

$$x = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}, W = \begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \\ w_{31} & w_{32} \end{bmatrix}$$

Then, define Affine layer, which is a matrix product. Get a 2 by 1(or 1 by 2) vector ($e$)

$$e = x\cdot W = x^T W = e^T $$

Also can be written in element,

$$\begin{bmatrix} e_1 & e_2 \end{bmatrix} = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix} \begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \\ w_{31} & w_{32} \end{bmatrix} = \begin{bmatrix} w_{11}x_1+w_{21}x_2+w_{31}x_3 & w_{12}x_1+w_{22}x_2+w_{32}x_3 \end{bmatrix}$$

Our propose is calculate $\dfrac{\partial L}{\partial W}$ to update weight matrix in $W_{new} \leftarrow W_{old} - \alpha \dfrac{\partial L}{\partial W_{old}}$.

Here $L$ is loss function doesn't matter what it is look like. 

$$\begin{aligned} \dfrac{\partial e}{\partial W} &= \dfrac{\partial e}{\partial vec(W)} \\
&= \begin{bmatrix} 
\dfrac{\partial e_1}{\partial w_{11}} & \dfrac{\partial e_2}{\partial w_{11}} \\
\dfrac{\partial e_1}{\partial w_{21}} & \dfrac{\partial e_2}{\partial w_{21}} \\
\dfrac{\partial e_1}{\partial w_{31}} & \dfrac{\partial e_2}{\partial w_{31}} \\
\dfrac{\partial e_1}{\partial w_{12}} & \dfrac{\partial e_2}{\partial w_{12}} \\
\dfrac{\partial e_1}{\partial w_{22}} & \dfrac{\partial e_2}{\partial w_{22}} \\
\dfrac{\partial e_1}{\partial w_{32}} & \dfrac{\partial e_2}{\partial w_{32}} \\
\end{bmatrix} \\
&= \begin{bmatrix} 
x_1 & 0 \\
x_2 & 0 \\
x_3 & 0 \\ \hdashline
0 & x_1 \\
0 & x_2 \\
0 & x_3 \\
\end{bmatrix} \\
&= \begin{bmatrix} 1 & 0 \\ 0 & 1\end{bmatrix} \otimes \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} \\ 
&= I(2) \otimes x
\end{aligned}$$

$$\begin{aligned} \dfrac{\partial L}{\partial W} &= \dfrac{\partial L}{\partial e} \dfrac{\partial e}{\partial W} \\
&= \begin{bmatrix} \dfrac{\partial L}{\partial e_1} & \dfrac{\partial L}{\partial e_2} \end{bmatrix} \cdot I(2) \otimes x \\
&= \begin{bmatrix} \dfrac{\partial L}{\partial e_1} & \dfrac{\partial L}{\partial e_2} \end{bmatrix} \cdot \begin{bmatrix} 1 & 0 \\ 0 & 1\end{bmatrix} \otimes \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} \\
&= \begin{bmatrix} \dfrac{\partial L}{\partial e_1} \\ \dfrac{\partial L}{\partial e_2} \end{bmatrix} \otimes \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} \\
&= \begin{bmatrix} 
\dfrac{\partial L}{\partial e_1} x_1 \\ 
\dfrac{\partial L}{\partial e_1} x_2 \\ 
\dfrac{\partial L}{\partial e_1} x_3 \\ 
\dfrac{\partial L}{\partial e_2} x_1 \\ 
\dfrac{\partial L}{\partial e_2} x_2 \\ 
\dfrac{\partial L}{\partial e_2} x_3 \end{bmatrix} = B \\ 
\end{aligned}$$

Becauese of vector transpose, $B$ is equal to below. 

$$\begin{aligned}
B &= B^{(3)} \rightarrow vector\ transpose\\
&= \begin{bmatrix} 
\dfrac{\partial L}{\partial e_1} x_1 & \dfrac{\partial L}{\partial e_2} x_1 \\
\dfrac{\partial L}{\partial e_1} x_2 & \dfrac{\partial L}{\partial e_2} x_2 \\ 
\dfrac{\partial L}{\partial e_1} x_3 & \dfrac{\partial L}{\partial e_2} x_3 \end{bmatrix} \\
&= \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} \cdot \begin{bmatrix} \dfrac{\partial L}{\partial e_1} & \dfrac{\partial L}{\partial e_2} \end{bmatrix} \\
\dfrac{\partial L}{\partial W} &= {(x^T)}^T \cdot \dfrac{\partial L}{\partial e}
\end{aligned}$$

Same for derivatives by $x$

$$\begin{aligned} \dfrac{\partial L}{\partial x} 
&= \dfrac{\partial L}{\partial e} \dfrac{\partial e}{\partial x} \\ 
&= \dfrac{\partial L}{\partial e} \cdot
\begin{bmatrix} 
\dfrac{\partial e_1}{\partial x_1} & \dfrac{\partial e_2}{\partial x_1} \\
\dfrac{\partial e_1}{\partial x_2} & \dfrac{\partial e_2}{\partial x_2} \\
\dfrac{\partial e_1}{\partial x_3} & \dfrac{\partial e_2}{\partial x_3} \end{bmatrix} \\
&= \dfrac{\partial L}{\partial e} \cdot \begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \\ w_{31} & w_{32} \end{bmatrix} \\
&= \dfrac{\partial L}{\partial e} \cdot W
\end{aligned}$$